In [4]:
pip install fosforml

     |████████████████████████████████| 51kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 1.9MB 14.5MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 74.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 84.0MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 86.7MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 143.0MB/s eta 0:00:01B 90.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 26.5MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 69.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 22.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 19.9MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 81.3MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 129.4MB/s eta 0:00:01
     |████████████████████████████████| 174kB 89.9MB/s eta 0:00:01
     |██████████████████████████████

In [5]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
pip install ydata-profiling --upgrade

     |████████████████████████████████| 358kB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 440kB 14.4MB/s eta 0:00:01
     |████████████████████████████████| 296kB 23.3MB/s eta 0:00:01
     |████████████████████████████████| 296kB 26.3MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 37.9MB/s eta 0:00:01
     |████████████████████████████████| 696kB 95.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 102.8MB/s eta 0:00:01
     |████████████████████████████████| 522kB 87.4MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 78.6MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 21.2MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 98.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 88.9MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 83.5MB/s eta 0:00:01
     |████████████████████████████████| 43.9MB 2.4MB/s eta 0:00:0101
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp39-n

In [ ]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
import seaborn as sns

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
df=sf_df.to_pandas()

In [ ]:
df.info()

In [ ]:
df.head()